In [1]:
import numpy as np
from scipy.io import loadmat

# Load the MATLAB file
data = loadmat('HMP_start_converted.mat')

# Extract relevant data from the loaded MATLAB file
abundances = data['abundances']
inds_species = data['inds_species'].flatten() - 1  # Assuming inds_species is 1-based in MATLAB
header = data['header']
img_ids = data['img_ids'].flatten()


# column of abundance means sample while row means species

# Step 1: Select rows of abundances based on inds_species
relabus = abundances[inds_species, :]

# Step 2: Remove rows where all elements are zero
inds2del = np.all(relabus == 0, axis=1)
relabus = relabus[~inds2del, :]

# choose data which we want to test
# stool samples phase 1 study
inds_samps = (header[4, :] == 'Stool') & (header[7, :] == '1')
relabus = relabus[:, inds_samps]

# Step 4: Remove corresponding img_ids elements
img_ids = img_ids[~inds2del]

# img_ids==0 means what?

# why total abundance less than 1?
coverage = np.sum(relabus[img_ids > 0, :], axis=0)
inds2del = coverage < 0.8
relabus = relabus[:, ~inds2del]

# Step 6: Replace NaNs with zeros
relabus = np.nan_to_num(relabus)


In [3]:
data

{'__header__': b'MATLAB 5.0 MAT-file, Platform: PCWIN64, Created on: Sun May 19 12:44:52 2024',
 '__version__': '1.0',
 '__globals__': [],
 'GCNs': array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [1, 0, 1, ..., 0, 0, 0]], dtype=uint8),
 'HMP_GCN_table': array([[1, 0, 0, ..., 0, 0, 0],
        [1, 0, 0, ..., 0, 0, 0],
        [0, 0, 1, ..., 0, 0, 0],
        ...,
        [1, 0, 1, ..., 0, 0, 0],
        [0, 0, 1, ..., 0, 0, 0],
        [0, 0, 1, ..., 0, 1, 0]], dtype=uint8),
 'HMP_IMGid': array([[array(['2502171170'], dtype='<U10')],
        [array(['2502171171'], dtype='<U10')],
        [array(['2502171173'], dtype='<U10')],
        ...,
        [array(['650716091'], dtype='<U9')],
        [array(['651324023'], dtype='<U9')],
        [array(['651324108'], dtype='<U9')]], dtype=object),
 'HMP_KOid': array([[array(['KO:K00001'], dtype='<U9')],
        [

In [35]:
import numpy as np

# Set random community parameters
N = 50
mm = 200
ps = np.arange(0.4, 0.9, 0.1)
dm = 0.5

# Assuming 'relabus' is a predefined numpy array
# Get empirical parameters
relabus_orig = relabus[np.random.choice(np.where(np.mean(relabus, axis=1) > 1e-6)[0], N, replace=False), :]
relabus_orig = relabus_orig / np.sum(relabus_orig, axis=0)
mm_orig = relabus.shape[1]

# Draw gene content
M = len(ps)
G = np.random.rand(N, M)
for j in range(M):
    G[:, j] = G[:, j] < ps[j]
G = G.astype(int)
G_orig = G.copy()
n1 = np.sum(G, axis=0)
n1_orig = n1.copy()
dm_origs = np.zeros(M)

# Draw species abundances
relabus = np.zeros((N, mm))
for k in range(mm):
    for i in range(N):
        relabus[i, k] = relabus_orig[i, np.random.randint(mm_orig)]
relabus = relabus / np.sum(relabus, axis=0)
mx = np.mean(relabus_orig, axis=1)

# Implement FB
TEs = np.array([1e-3, 5e-3, 1e-2, 0.05, 0.1, 1])
TEs = np.flip(TEs)
reps = int(1e2)
G_fb = np.zeros_like(G_orig)  # Initialize G_fb with the same shape as G_orig
for j in range(M):
    n1 = n1_orig[j]
    inds_perm = np.random.permutation(N)[:n1]
    G_col = np.zeros(N, dtype=int)
    G_col[inds_perm] = 1
    dmthis = np.log10(np.mean(mx[G_col == 1])) - np.log10(np.mean(mx[G_col == 0]))
    for tthis in range(len(TEs)):
        TE = dm * TEs[tthis]
        for rep in range(reps):
            ind2switch = np.random.randint(n1)
            inds_out = np.where(G_col == 0)[0]
            ind2 = np.random.choice(inds_out, 1)[0]
            inds_new = inds_perm.copy()
            inds_new[ind2switch] = ind2
            Gnew_col = np.zeros(N, dtype=int)
            Gnew_col[inds_new] = 1
            dmnew = np.log10(np.mean(mx[Gnew_col == 1])) - np.log10(np.mean(mx[Gnew_col == 0]))
            Ethis = np.abs(dmthis - dm)
            Enew = np.abs(dmnew - dm)
            dE = Enew - Ethis
            pkeep = np.exp(-dE / TE)
            if np.random.rand() < pkeep:
                inds_perm = inds_new
                dmthis = dmnew
                G_col = Gnew_col
    G_fb[:, j] = G_col
    dm_origs[j] = dmthis


# Implement FR
G = G_fb
gall = np.dot(G.T, relabus)


[20 23 29 29 39]
[0.49975122 0.49976674 0.50110156 0.49917723 0.50004488]


In [10]:
data

{'__header__': b'MATLAB 5.0 MAT-file, Platform: PCWIN64, Created on: Sun May 19 12:44:52 2024',
 '__version__': '1.0',
 '__globals__': [],
 'GCNs': array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [1, 0, 1, ..., 0, 0, 0]], dtype=uint8),
 'HMP_GCN_table': array([[1, 0, 0, ..., 0, 0, 0],
        [1, 0, 0, ..., 0, 0, 0],
        [0, 0, 1, ..., 0, 0, 0],
        ...,
        [1, 0, 1, ..., 0, 0, 0],
        [0, 0, 1, ..., 0, 0, 0],
        [0, 0, 1, ..., 0, 1, 0]], dtype=uint8),
 'HMP_IMGid': array([[array(['2502171170'], dtype='<U10')],
        [array(['2502171171'], dtype='<U10')],
        [array(['2502171173'], dtype='<U10')],
        ...,
        [array(['650716091'], dtype='<U9')],
        [array(['651324023'], dtype='<U9')],
        [array(['651324108'], dtype='<U9')]], dtype=object),
 'HMP_KOid': array([[array(['KO:K00001'], dtype='<U9')],
        [